In [3]:
import pandas as pd
import numpy as np
import glob
import cv2
import os
import random
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn import preprocessing
from scipy import stats
from sklearn.model_selection import GridSearchCV, KFold
from math import sqrt
from sklearn.linear_model import SGDRegressor, LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import precision_score, recall_score, f1_score, hamming_loss
import json

In [4]:
directory = './regression_modeldata1'

with open('groundtruth.json') as json_file:
    gtdata = json.load(json_file)
subdirs = os.listdir(directory)

finalX = []
finalyLeft = []
finalyRight = []

for i in range(len(subdirs)):
    dir2pull = directory + '/' + subdirs[i] + '/'
    
    lefty = gtdata[subdirs[i]]['leftlength']
    righty = gtdata[subdirs[i]]['rightlength']
    
    
    for file in glob.glob(dir2pull + '*.csv'):
        data2read = file.replace("\\",'/')
        #print(data2read)
        singledata = pd.read_csv(data2read,header=None)
        singledata = np.array(singledata)
        
        dataarray = []
        
        for j in range(singledata.shape[1]):
            if isinstance(singledata[0,j],str):
                if singledata[0,j][0:2] == 'SI' or singledata[0,j][0:2]==' S':
                    datapoint = singledata[0,j].split('(')[1]
                elif singledata[0,j][0:2] == 'L ' or singledata[0,j][0:2] == 'R ':
                    datapoint = singledata[0,j][2:]
                else:
                    datapoint = singledata[0,j].split(')')[0]
                datapoint = float(datapoint)
            else:
                datapoint = singledata[0,j]
            dataarray.append(datapoint)
        procdata = np.array(dataarray)
        #print(procdata.shape)
        
        if procdata.shape[0] != 60:
            continue
        
        finalX.append(procdata)
        finalyLeft.append(lefty + random.uniform(-0.5,0.5))
        finalyRight.append(righty + random.uniform(-0.5,0.5))
#print(len(finalX))

finalX = np.array(finalX)
print(finalX.shape)

finalyLeft = np.array(finalyLeft)

X = finalX
#np.random.shuffle(X)
y = finalyLeft
def shuffle_in_unison(a, b):
    n_elem = a.shape[0]
    indeces = np.random.choice(n_elem, size=n_elem, replace=False)
    return a[indeces], b[indeces]
    
X,y = shuffle_in_unison(X,y)


(123, 60)


In [5]:
# with normalized features

cv = KFold(n_splits=10,shuffle=True,random_state=234)
for train_index, test_index in cv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # can comment out break and bring training into loop for cross validation
    break

print(X_train.shape)
print(X_test.shape)
    
# standard scaler used to z-score
normalizer = preprocessing.StandardScaler().fit(X_train)
Xpreproc_train = normalizer.transform(X_train)
Xpreproc_test = normalizer.transform(X_test)

# Now use scikit-learn's random forest regressor to train the model using
# the parameters pulled from above
#model = LinearRegression().fit(Xpreproc_train,y_train)
model = RandomForestRegressor().fit(Xpreproc_train,y_train)
#model = GradientBoostingRegressor().fit(Xpreproc_train,y_train)
model = model.fit(Xpreproc_train,y_train)

# validation metrics calculated here to examine model performance
ypred_train = model.predict(Xpreproc_train)
ypred_test = model.predict(Xpreproc_test)


train_rmse = sqrt(mean_squared_error(y_train[:], ypred_train[:]))
test_rmse = sqrt(mean_squared_error(y_test[:],ypred_test[:]))
train_mae = mean_absolute_error(y_train[:], ypred_train[:])
test_mae = mean_absolute_error(y_test[:],ypred_test[:])

print('Train RMSE {}, Train MAE {}, Test RMSE {}, Test MAE {}'.format(train_rmse,train_mae,test_rmse,test_mae))

(110, 60)
(13, 60)
Train RMSE 0.22635762833350362, Train MAE 0.13548862863503308, Test RMSE 0.8493179947735384, Test MAE 0.5084286745229835


In [6]:
cv = KFold(n_splits=10,shuffle=True,random_state=234)
for train_index, test_index in cv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # can comment out break and bring training into loop for cross validation
    break

print(X_train.shape)
print(X_test.shape)
    

# Now use scikit-learn's random forest regressor to train the model using
# the parameters pulled from above
#model = LinearRegression().fit(Xpreproc_train,y_train)
#model = RandomForestRegressor().fit(X_train,y_train)
model = GradientBoostingRegressor().fit(Xpreproc_train,y_train)
model = model.fit(X_train,y_train)

# validation metrics calculated here to examine model performance
ypred_train = model.predict(X_train)
ypred_test = model.predict(X_test)


train_rmse = sqrt(mean_squared_error(y_train[:], ypred_train[:]))
test_rmse = sqrt(mean_squared_error(y_test[:],ypred_test[:]))
train_mae = mean_absolute_error(y_train[:], ypred_train[:])
test_mae = mean_absolute_error(y_test[:],ypred_test[:])

print('Train RMSE {}, Train MAE {}, Test RMSE {}, Test MAE {}'.format(train_rmse,train_mae,test_rmse,test_mae))

# save model to file
from sklearn.externals import joblib
joblib_file = "ARLengthModel.pkl"
joblib.dump(model, joblib_file)

(110, 60)
(13, 60)
Train RMSE 0.060190380125191414, Train MAE 0.048381828919403604, Test RMSE 0.6912601945728785, Test MAE 0.44120565222246894


/Users/vaneesh_k/opt/anaconda3/envs/tf2/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['ARLengthModel.pkl']

In [7]:
# not tested on an apple computer, may need adjustment

import coremltools

coreml_model = coremltools.converters.sklearn.convert(model)
coreml_model.save('ardata2length.mlmodel')

ImportError: cannot import name 'Imputer' from 'sklearn.preprocessing' (/Users/vaneesh_k/opt/anaconda3/envs/tf2/lib/python3.7/site-packages/sklearn/preprocessing/__init__.py)